In [ ]:
import cv2
import numpy as np

In [ ]:
img_folder = "E:/Data/deepfasion/train_test_256/train"
pth_folder = "E:/Data/deepfasion/train_test_256/pth_train"

1. 循环加载并读取所有图片，然后检查图片的尺寸是否全部一致
2. 在接下来搭建神经网络训练的流水线时，需要加载图像，然后把图像转化成torch.Tensor，但是如果图像原封不动为jpg格式，那么每次加载都要加载jpg->转换成numpy->转换成cuda Tensor。并且我们会训练多个epoch，每个epoch所有图片都要像这里说的这样处理一遍，训练E个epoch就要加载、转换E遍。所以可以先全把图像加载了，转换成torch.Tensor然后存成.pth文件。每次一加载直接是torch.Tensor，只需要cuda一下就好了。在此过程中，可以先把normalization等等预处理步骤先做了，这样加载图像时，这个tensor直接是可以送进神经网络的状态 (torch.Tensor, float32)。因为要考虑内存能不能放得下。如果能放下，就直接存成一个文件。如果放不下，意味着内存不支持直接读取这个数据集文件，可以每个图片存一个.pth文件，.pth文件和.jpg文件的区别在于.pth存储的是一个预处理后的torch.Tensor。
3. 文件夹Dataset存储所有数据相关的文件，完成数据加载器DFCDataset.py
4. 文件夹Models存储神经网络相关的文件
5. 文件夹Runs用来存储将来可能会有的训练日志以及训练存的模型

In [4]:
# 总共需要多少存储空间？
n_imgs = 37210
n_floats_per_img = 192 * 256 * 3
n_bytes_per_float = 4

n_bytes = n_imgs * n_floats_per_img * n_bytes_per_float

print(f"Storage need {n_bytes/1024/1024/1024} GB")

Storage need 20.4400634765625 GB


In [3]:
n_floats_per_img * n_bytes_per_float / 1024 / 1024

0.5625

In [ ]:
# 循环目录里的所有图片

# 把图片转换成torch.Tensor, float32, (C, H, W), 取值-1到1区间格式

# 保存到 10位id表示衣服_该衣服的第几张图片.pth 文件